Let's train a PET classifier

In [2]:
from fastai.vision.all import *

path = untar_data(URLs.PETS)




In [5]:
dls = ImageDataLoaders.from_name_re(
    path, get_image_files(path/'images'),
    pat='(.+)_\d+.jpg',
    item_tfms=Resize(230),
    batch_tfms=aug_transforms(size=224, min_scale=0.75)
)

learn = vision_learner(dls, models.resnet18, metrics=accuracy)
learn.fine_tune(1)
learn.path = Path('.')


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/chris/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,accuracy,time
0,1.629318,0.430374,0.861299,06:21


epoch,train_loss,valid_loss,accuracy,time
0,0.555940,0.301212,0.899188,08:16


In [6]:
learn.export()

Let's load the exported model and run in gradio

In [7]:
from fastai.vision.all import *
learn = load_learner('export.pkl')

In [8]:
labels = learn.dls.vocab
def predict(img):
    img = PILImage.create(img)
    pred,pred_idx,probs = learn.predict(img)
    return {labels[i]: float(probs[i]) for i in range(len(labels))}

In [15]:
# !pip install gradio


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -y


In [17]:
import gradio as gr

gradio_interface = gr.Interface(
    fn=predict,
    inputs=gr.inputs.Image(shape=(512, 512)),
    outputs=gr.outputs.Label(num_top_classes=3),
    examples=["cat.jpeg", "dog.jpeg", "catdog.jpeg"]
)
gradio_interface.launch()


/opt/homebrew/Caskroom/miniconda/base/envs/nuefast/lib/python3.8/site-packages/gradio/inputs.py:256: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/nuefast/lib/python3.8/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/opt/homebrew/Caskroom/miniconda/base/envs/nuefast/lib/python3.8/site-packages/gradio/outputs.py:196: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/nuefast/lib/python3.8/site-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://55990d7fe854e229.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x17e312040>,
 'http://127.0.0.1:7861/',
 'https://55990d7fe854e229.gradio.app')